In [ ]:
import pandas as pd

In [ ]:
class BacktestBollingerBands:
    def __init__(self) -> None:
        self.df = pd.DataFrame()

    def load_data(self, path):
        self.df = pd.read_csv(path)
        self.df["date"] = pd.to_datetime(self.df["date"], unit="ms")
        self.df = self.df.set_index(self.df["date"])
        del self.df["date"]

    def populate_indicators(self):
        self.df['BB_MID'] = self.df['close'].rolling(20).mean() 
        rolling_std = self.df['close'].rolling(20).std()
        self.df['BB_UP'] = self.df['BB_MID'] + 2 * rolling_std
        self.df['BB_LOW'] = self.df['BB_MID'] - 2 * rolling_std


    def populate_signals(self):
        self.df['buy_signal'] = False
        self.df['sell_signal'] = False
        self.df.loc[(self.df['close'] > self.df['BB_UP']), 'buy_signal'] = True
        self.df.loc[(self.df['close'] < self.df['BB_MID']), 'sell_signal'] = True

    def run_backtest(self):
        balance = 1000
        position = None
        asset = "BTC"

        for index, row in self.df.iterrows():

            if position is None and row['buy_signal']:
                open_price = row['close']
                position = {
                    'open_price': open_price,
                    'usd_size': balance,
                }
                print(f"{index} - Buy for {balance}$ of {asset} at {open_price}$")

            elif position and row['sell_signal']:
                close_price = row['close']
                trade_result = (close_price - position['open_price']) / position['open_price']
                balance = balance + trade_result * position['usd_size']
                position = None
                print(f"{index} - Sell for {balance}$ of {asset} at {close_price}$")
                
            
        
        print(f"Final balance: {balance}$")


In [ ]:
bt = BacktestBollingerBands()
bt.load_data('BTC-USDT.csv')
bt.populate_indicators()
bt.populate_signals()
bt.run_backtest()